In [1]:
import os
# import math
import numpy as np
import shapely as shp
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio as rio
import rasterio.mask

In [2]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\ETM Images Raw\\Havard"
cwd_Images_Processed = cwd + "\\ETM Images Processed\\Havard"

In [3]:
b1 = rio.open(cwd_Images_Raw + "\\LE07_L2SP_013030_20020908_20200916_02_T1_SR_B1.TIF").read(1)
b2 = rio.open(cwd_Images_Raw + "\\LE07_L2SP_013030_20020908_20200916_02_T1_SR_B2.TIF").read(1)
b3 = rio.open(cwd_Images_Raw + "\\LE07_L2SP_013030_20020908_20200916_02_T1_SR_B3.TIF").read(1)
b4 = rio.open(cwd_Images_Raw + "\\LE07_L2SP_013030_20020908_20200916_02_T1_SR_B4.TIF").read(1)
b5 = rio.open(cwd_Images_Raw + "\\LE07_L2SP_013030_20020908_20200916_02_T1_SR_B5.TIF").read(1)
b7 = rio.open(cwd_Images_Raw + "\\LE07_L2SP_013030_20020908_20200916_02_T1_SR_B7.TIF").read(1)

0.356a1 + 0.13a3 + 0.373a4 + 0.085a5 + 0.072a7 - 0.0018

In [4]:
# albedo = 0.356 * b1 + 0.13 * b3 + 0.373 * b4 + 0.085 * b5 + 0.072 * b7 - 0.0018
albedo = 0.526 * b2 + 0.3139 * b4 + 0.112 * b7

In [5]:
src = rio.open(cwd_Images_Raw + "\\LE07_L2SP_013030_20020908_20200916_02_T1_SR_B1.TIF")
out_meta = src.meta
out_meta.update({
    "driver": "GTiff",
    "dtype": 'float64'
})
with rio.open(cwd_Images_Processed + "\\Albedo.tif", 'w', **out_meta) as dest:
    dest.write(albedo, 1)

In [6]:
# Create a point shapefile based on the site, using Lon-Lat
gdf_4326 = gpd.GeoDataFrame(
    pd.DataFrame({
        "Site": ["Havard Forest"]
    }),
    geometry = gpd.points_from_xy([-72.171], [42.538]),
    crs = "EPSG:4326"
)
gdf_4326

,Site,geometry
0,Havard Forest,POINT (-72.17100 42.53800)


In [7]:
gdf_32618 = gdf_4326.to_crs("EPSG:32618")

In [8]:
# First we retrieve the x, y coordinate of our site
site_x = gdf_32618.geometry.x.values[0]
site_y = gdf_32618.geometry.y.values[0]
print("Site X, Y: " + str(site_x) + "," + str(site_y))

Site X, Y: 732315.442750216,4713391.750567996


In [9]:
# Then we get the row, col of our site in the raster, so that we know the pixel the site belongs to
site_row, site_col = rio.open(cwd_Images_Raw + "\\LE07_L2SP_013030_20020908_20200916_02_T1_SR_B1.TIF").index(site_x, site_y)
print("Site falls on the pixel: " + str(site_row) + ", " + str(site_col))

Site falls on the pixel: 5907, 4801


In [10]:
# Now we get the spatial coordinates of that pixel (coordinates of its center)
site_pixel_x, site_pixel_y = rio.open(cwd_Images_Raw + "\\LE07_L2SP_013030_20020908_20200916_02_T1_SR_B1.TIF").xy(site_row, site_col)
print("The spatial coordinates of the (center) pixel which the site falls on is: " + str(site_pixel_x) + ", " + str(site_pixel_y))

The spatial coordinates of the (center) pixel which the site falls on is: 732330.0, 4713390.0


In [11]:
site_x_left_1000m = site_pixel_x - 510
site_x_right_1000m = site_pixel_x + 510
site_y_top_1000m = site_pixel_y + 510
site_y_bottom_1000m = site_pixel_y - 510

In [12]:
shp_10m = shp.box(site_x_left_1000m, site_y_bottom_1000m, site_x_right_1000m, site_y_top_1000m)
gdf_10m = gpd.GeoDataFrame(
    pd.DataFrame({"0": ["0"]}),
    geometry=[shp_10m],
    crs = "EPSG:32618"
)

In [13]:
src = rio.open(cwd_Images_Processed + "\\Albedo.tif")
out_image, out_transform = rio.mask.mask(src, gdf_10m.geometry, crop=True)
out_meta = src.meta
out_meta.update({"driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform})

with rio.open(cwd_Images_Processed + "\\Albedo 1000m.tif", "w", **out_meta) as dest:
    dest.write(out_image)